# 国际化文本翻译

我们写app希望可以适应本地化需求,也就是当换一种语言的时候可以自动转成翻译好的对应文本.我们当然可以每个语言些一个版本,代码相同只是修改其中的文本.

一个简单的解决方案是使用一个函数包裹字符串,让函数负责找到对应翻译.比如


In [1]:

spanishStrings = {'Hello world!': 'Hola Mundo!'}
frenchStrings = {'Hello world!': 'Bonjour le monde!'}
germanStrings = {'Hello world!': 'Hallo Welt!'}
  

In [2]:
def trans(s):
    if LANGUAGE == 'English':
        return s
    if LANGUAGE == 'Spanish':
        return spanishStrings.get(s)
    if LANGUAGE == 'French':
        return frenchStrings.get(s)
    if LANGUAGE == 'German':
        return germanStrings.get(s)

In [3]:
LANGUAGE = 'French'
print(trans("Hello world!"))

Bonjour le monde!


但是很明显,一旦文本量变大了就会无法管理了~

Python提供了gettext模块用于解决这类问题


## gettext的使用

> 创建国际化文档的文件夹目录


    ----|
        |-src-|
              |-locale-|
                       |-en-|
                       |    |-LC_MESSAGES
                       |
                       |-cn-|
                       |    |-LC_MESSAGES
                       |
                       |-fr-|
                            |-LC_MESSAGES
        

> gettext初始化

使用脚本工具`pygettext`初始化gettext设置(如果安装的python中没有的话可以来[这里下载](./src/pygettext.py))

In [4]:
!src/pygettext.py -p src/

  File "src/pygettext.py", line 516
    except getopt.error, msg:
                       ^
SyntaxError: invalid syntax


In [5]:
!cat src/messages.pot

# SOME DESCRIPTIVE TITLE.
# Copyright (C) YEAR ORGANIZATION
# FIRST AUTHOR <EMAIL@ADDRESS>, YEAR.
#
msgid ""
msgstr ""
"Project-Id-Version: PACKAGE VERSION\n"
"POT-Creation-Date: 2016-12-08 20:34+CST\n"
"PO-Revision-Date: YEAR-MO-DA HO:MI+ZONE\n"
"Last-Translator: FULL NAME <EMAIL@ADDRESS>\n"
"Language-Team: LANGUAGE <LL@li.org>\n"
"MIME-Version: 1.0\n"
"Content-Type: text/plain; charset=CHARSET\n"
"Content-Transfer-Encoding: ENCODING\n"
"Generated-By: pygettext.py 1.5\n"




我们修改它的

"Content-Type: text/plain; charset=CHARSET\n"
"Content-Transfer-Encoding: ENCODING\n"

两个字段,并为其添加要翻译的内容

In [6]:
%%writefile src/transfor.pot

# SOME DESCRIPTIVE TITLE.
# Copyright (C) YEAR ORGANIZATION
# FIRST AUTHOR <EMAIL@ADDRESS>, YEAR.
#
msgid ""
msgstr ""
"Project-Id-Version: PACKAGE VERSION\n"
"POT-Creation-Date: 2016-12-08 20:34+CST\n"
"PO-Revision-Date: YEAR-MO-DA HO:MI+ZONE\n"
"Last-Translator: FULL NAME <EMAIL@ADDRESS>\n"
"Language-Team: LANGUAGE <LL@li.org>\n"
"MIME-Version: 1.0\n"
"Content-Type: text/plain; charset=gb2312\n"
"Content-Transfer-Encoding: utf-8\n"
"Generated-By: pygettext.py 1.5\n"


msgid "hello"
msgstr ""

msgid "Python now"
msgstr ""


Overwriting src/transfor.pot


接着我们就可以使用[poedit](https://poedit.net/)来逐条翻译了这边有一个[基本教程操作](http://teliute.laxjyj.com/TeacHtm/TePoedit/index.html)

我们用`poedit`为写一份中文的翻译,放在locale/cn/LC_MESSAGES中,其中包含两份文件,zh_CN.po和zh_CN.mo,同样的也弄一份英文的

In [7]:
!cat src/locale/cn/zh_CN.po

cat: src/locale/cn/zh_CN.po: No such file or directory


> 注册国际化文本

In [8]:
%%writefile src/transfer.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import gettext
langen = gettext.translation('en', './src/locale', languages=['en'])
langcn = gettext.translation('zh_CN', './src/locale', languages=['cn'])

Overwriting src/transfer.py


其中:

+ `gettext_te.py`是要翻译模块或app名
+ `./locale`是存放翻译文件的路径,
+ `languages`参数指定要使用的语言存放的子目录,这里cn表示使用`./locale/cn/LC_MESSAGES/`路径下的翻译文件.

这样我们就有了一个`_()`方法来翻译文本

> 编辑主模块

In [9]:
%%writefile src/gettext_te.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
from transfer import *
langcn.install()
print(_("Hello world!"))
langen.install()
print(_("Hello world!"))

Overwriting src/gettext_te.py


In [10]:
%run src/gettext_te.py

Hello world!
Hello world!


这样每次只要修改对应文件夹的`mo`文件就可以实现本地化翻译了